# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt


Summaries_file = 'recognition_Summaries.pkl.bz2'
Abstracts_file = 'recognition_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(list)

for id in sorted(Summaries.keys()):
    term_set = set(preprocess(tokenize(Summaries[id].title)))
    if id in Abstracts:
        term_set.update(preprocess(tokenize(Abstracts[id])))
    for term in term_set:
        inverted_index[term].append(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [5]:
! pip install --user nltk

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
stemmer = EnglishStemmer()

s = '''Good muffins cost $4.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

INPUT TEXT:
  Good muffins cost $4.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$4.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '4.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaspreetsingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaspreetsingh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jaspreetsingh/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jaspreetsingh/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [7]:
print(stemmer.stem("processes"))

process


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)

for doc_id in Summaries.keys():
    tokens = preprocess(tokenize(Summaries[doc_id].title))
    if (doc_id in Abstracts):
        tokens.extend(preprocess(tokenize(Abstracts[doc_id])))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Summaries))

Let's test these functions with some examples:

In [9]:
print(tf('enzyme', 33055208))
print(df('enzyme'))
print(num_documents)

1.0
11027.0
481216.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

**Your name:** Jaspreet Singh

### Task 1

Implement in the code block below the `smart_tokenize_and_preprocess` function using NLTK's functions for tokenization and stemming. 

In [10]:
# Smarter linguistic processing

def smart_tokenize_and_preprocess(text):
    tokenize = word_tokenize(text)
    #doc_words = [word.lower() for word in tokenize if word.isalpha()]  this could be used to get rid of , . ! $
    stem = [stemmer.stem(w) for w in tokenize]
    return stem
    

# To test it:
print(smart_tokenize_and_preprocess("Books about Information Retrieval (IR) etc. cost at least $25.00!"))

['book', 'about', 'inform', 'retriev', '(', 'ir', ')', 'etc', '.', 'cost', 'at', 'least', '$', '25.00', '!']


Now we can make a smarter index based on this function. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [11]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(list)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = list(key for key in Summaries.keys() if 33000000 <= key < 34000000)

# Building our smarter index:
for id in sorted(subset_of_ids):
    term_set = set(smart_tokenize_and_preprocess(Summaries[id].title))
    if id in Abstracts:
        term_set.update(smart_tokenize_and_preprocess(Abstracts[id]))
    for term in term_set:
        smarter_index[term].append(id)

Now implement the `smarter_and_query` function, based on the function `smarter_tokenize_and_preprocess` you defined above and accessing our new index `smarter_index`. You can start from copying the code for `and_query` from the last assignment. For that to work, you'll also have to copy the code for the `and_merge` function from the last assignment.

In [20]:
# Smarter and_query based on the smarter tokenize and preprocess functions

def and_merge(sorted_list1, sorted_list2):  
    merged_list= []
    index1 = 0
    index2 = 0
    size1 = len(sorted_list1)
    size2 = len(sorted_list2)
    seen_all_index = False
    while seen_all_index == False:
        if((size1 != index1) and (size2 != index2) ):
            if sorted_list1[index1] == sorted_list2[index2]:
                merged_list.append(sorted_list1[index1])
                index1 +=1
                index2 +=1
                continue
            if(sorted_list1[index1]>sorted_list2[index2]):
                index2 +=1
                continue
            if(sorted_list2[index2]>sorted_list1[index1]):
                index1 +=1
                continue
        else:
            seen_all_index = True
    return merged_list  

def smarter_and_query(query_string):  
    preproccesedTokens = smart_tokenize_and_preprocess(query_string)

    posting_lists = []

    for token in preproccesedTokens:
        if(token in inverted_index):
            posting_lists.append(smarter_index[token])

    and_list = posting_lists[0]
    for i in range(1, len(posting_lists)):
        and_list = and_merge(and_list, posting_lists[i])
    return and_list  

### Task 2

Run the query "mutation of enzyme complexes" with the new `smarter_and_query` function from task 1. Does it return paper *33055208*? Explain what our new smarter function specifically contributes to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [21]:
print(smart_tokenize_and_preprocess("mutation of enzyme complexes"))
smarter_and_query("mutation of enzyme complexes")  


['mutat', 'of', 'enzym', 'complex']


[33014341,
 33014620,
 33023230,
 33037049,
 33040723,
 33048983,
 33052948,
 33053321,
 33055208,
 33056978,
 33057400,
 33058871,
 33068261,
 33070870,
 33092221,
 33107822,
 33122294,
 33125859,
 33126897,
 33127861,
 33152247,
 33172890,
 33176178,
 33196685,
 33202035,
 33206170,
 33235207,
 33253435,
 33264373,
 33291532,
 33295760,
 33300026,
 33304400,
 33323411,
 33340646,
 33357431,
 33386150,
 33398001,
 33401921,
 33428378,
 33434574,
 33441663,
 33450351,
 33453165,
 33525768,
 33528563,
 33532768,
 33554132,
 33565573,
 33574254,
 33583326,
 33586135,
 33588726,
 33600043,
 33630857,
 33636637,
 33639170,
 33651967,
 33655245,
 33663878,
 33665744,
 33668198,
 33676181,
 33681704,
 33682442,
 33689684,
 33708793,
 33712450,
 33729394,
 33730245,
 33754021,
 33775828,
 33779749,
 33784401,
 33784856,
 33792302,
 33836029,
 33838354,
 33840529,
 33863288,
 33876824,
 33888601,
 33893722,
 33921405,
 33940160,
 33945229,
 33960519,
 33969420,
 33972509,
 33978746,
 33981015,

In [22]:
display_summary(33055208)

**Answer:** 
Our preprocessing (old) function previously did not make use of stemming, but as we have now implemented in our smarterpreprocess which uses a stemmer. This reduces words like "mutation" to "mutat" and "complexes" to "complex," as they share the same meaning. In dataset 33055208,  "mutation" did not occur however 'mutations' did, it has now been shortened to "mutat" by the smarterpreprocess and now we can also capture the word mutation in our smarter_and_query. Similarly for, "complexes," which did not occur in 33055208, however "complex" and "complexed" occur 33055208. With our smarter preprocess it is now stemmed to "complex," resulting in its appearance in our smarter_and_query.

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. Use the _add-one-smoothing_ version of idf, so we don't run into problems with terms that don't appear in the collection at all. The relevant formulas can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [23]:

def idf(t):
    return float(log10((num_documents+1)/(df(t)+1)))

def tfidf(t,d):
    return tf(t,d)*idf(t)
    



print(tfidf('mutation', 33055208))
print(tfidf('of', 33055208))
print(tfidf('enzyme', 33055208))
print(tfidf('complexes', 33055208))

0.0
0.14123984973343998
1.6398442038686944
0.0


### Task 4

Create a function `query_ntn_nnn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn.nnn`, as above (see the formula for the `ntn.nnn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn_nnn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your functions `or_merge` and `or_query` from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above.

Demonstrate your function by giving it the exemplary query string "mutation of enzyme complexes".

In [24]:
def merge_sorted_or(sortedList1, sortedList2):
    mergeSortedList= []
    index1 = 0
    index2 = 0
    size1 = len(sortedList1)
    size2 = len(sortedList2)
    seen_all_index = False
    while seen_all_index == False:
        if((size1 != index1) and (size2 != index2) ):
            if sortedList1[index1] == sortedList2[index2]:
                mergeSortedList.append(sortedList1[index1])
                index1 +=1
                index2 +=1
                continue
            if(sortedList1[index1]>sortedList2[index2]):
                mergeSortedList.append(sortedList2[index2])
                index2 +=1
                continue
            if(sortedList2[index2]>sortedList1[index1]):
                mergeSortedList.append(sortedList1[index1])
                index1 +=1
                continue
        
        else:
            if(size1 != index1):
                mergeSortedList.append(sortedList1[index1])
                index1+=1
                continue
            if(size2 != index2):
                mergeSortedList.append(sortedList2[index2])
                index2+=1
                continue
            else:
                seen_all_index = True
    return mergeSortedList
            

def query_or(input_query):
    preproccesedTokens = preprocess(tokenize(input_query))
    
    posting_lists = []

    for token in preproccesedTokens:
        if(token in inverted_index):
            posting_lists.append(inverted_index[token])
            
    currentQueryOr = posting_lists[0]
    for i in range(1, len(posting_lists)):
        currentQueryOr = merge_sorted_or(currentQueryOr, posting_lists[i])
        
    return currentQueryOr

def score_ntn_nnn(tokens,d):
    score = 0
    for t in tokens:
        score += tfidf(t,d)
    return score
    
    

def query_ntn_nnn(query_string):
    scoredoc = {}
    paper_or_docs = query_or(query_string)
    preproccesedTokens = preprocess(tokenize(query_string))
    for d in paper_or_docs:
        scoredoc[d] =  score_ntn_nnn(preproccesedTokens,d)
    sorted_scoredoc = sorted(scoredoc.items(), key=lambda x:x[1], reverse = True)
    
    top_10 = sorted_scoredoc[:10]
    for q, s in top_10:
        print(f"Document ID: {q}, Score: {s}")
            
        
    
    

In [25]:
# Example query:
query_ntn_nnn("mutation of enzyme complexes")

Document ID: 39073830, Score: 29.607075574012324
Document ID: 17264471, Score: 22.553644894234456
Document ID: 7165718, Score: 21.561934390741698
Document ID: 10503276, Score: 21.523414431723484
Document ID: 16666772, Score: 21.45921450002647
Document ID: 1562739, Score: 20.421708296058004
Document ID: 28270751, Score: 20.324470890964882
Document ID: 26448454, Score: 20.285950931946672
Document ID: 11234382, Score: 20.126113491841952
Document ID: 11811950, Score: 18.46776339578783


### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc_ntc`. This second version should use, as its name suggests, variant `ntc.ntc` instead of `ntn.nnn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc.nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.)

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [26]:
tfidf_length_values = defaultdict(int)

for doc_id in Summaries.keys():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc_ntc` function with the same example query as above: "mutation of enzyme complexes"

In [29]:
# Your code here:
def score_ntc_ntc(tokens,d, weightq):
    scoreDoc = 0
    tfidf_q_length = 0
    for token in tokens:
        if(token in tf_matrix[d]):
            tfidf_q_length += weightq[token] ** 2
            scoreDoc += weightq[token] *tfidf(token, d)
    if scoreDoc == 0 or tfidf_q_length == 0:
        return 0
    return scoreDoc/(tfidf_length(d)* tfidf_q_length**(0.5))

def query_ntc_ntc(q):
    scoresDoc = {}
    postinglist = query_or(q)
    for d in postinglist:
        scoresDoc[d] = 0
    preproccesedTokens = preprocess(tokenize(q))
    weightst_q = {}
    for token in preproccesedTokens:
        weightst_q[token] = idf(token)
    for d in postinglist:
        scoresDoc[d] += score_ntc_ntc(preproccesedTokens,d, weightst_q)


    sorted_scoredoc = sorted(scoresDoc.items(), key=lambda x:x[1], reverse = True)
    
    top_10 = sorted_scoredoc[:10]
    for q, s in top_10:
        print(f"Document ID: {q}, Score: {s}")
        display_summary(q)
        
        

# Example query:
query_ntc_ntc("mutation of enzyme complexes")

Document ID: 34513841, Score: 0.5297682864247942


Document ID: 39073830, Score: 0.5288339481880096


Document ID: 36342236, Score: 0.5068926288893287


Document ID: 17327177, Score: 0.4924810916867742


Document ID: 6967885, Score: 0.4811811294676484


Document ID: 10084112, Score: 0.4352568229882278


Document ID: 10503276, Score: 0.4345217851712705


Document ID: 16337168, Score: 0.43301533342012316


Document ID: 35685137, Score: 0.429952610485576


Document ID: 38196066, Score: 0.41789739277940524


# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done **individually**, and that code sharing or copying are **strictly forbidden** and will be punished.